<center style="font-size:2em;font-weight:bold">
   Simple Example
</center>
<p/>
<center style="font-size:1.5em;font-weight:bold">
   of Interactive Drawing and Interactive Plot
</center>
<p/>
<center>Thomas E. Vaughan</center>

<!--- vim: set filetype=markdown tw=75: -->



# Configuration of Notebook

The following javascript-magic cell enables the editing of any cell in the
external editor, gvim.

<!--- vim: set filetype=markdown tw=75 -->



In [ ]:
%%javascript

// This cell defines two command-mode shortcuts.
//
// Pressing 'g' in command mode copies the content of the current cell into
// a file and then launches gvim to edit that file.
//
// Pressing 'u' in command mode updates the current cell with the contents
// of the file.
//
// The idea is first to press 'g', then to edit the file with gvim, then to
// exit from gvim, and finally to press 'u' to update the cell from the
// file.

// The 'g' shortcut.
IPython.keyboard_manager.command_shortcuts.add_shortcut('g', {
    handler : function (event) {
        var input = IPython.notebook.get_selected_cell().get_text();
        var cmd = "f = open('.toto.txt', 'w');f.close()";
        if (input != "") {
            cmd = '%%writefile .toto.txt\n' + input;
        }
        IPython.notebook.kernel.execute(cmd);
        cmd = "import os;"
        // Establish a clean PATH that has only operating-system-native
        // executables.  On my machine, having '/opt/miniconda3/bin' in the
        // PATH, as it is by default, causes youcompleteme to malfunction
        // because it finds '/opt/miniconda3/bin/python' before
        // '/usr/bin/python'.
        cmd = cmd + "os.putenv('PATH','/bin:/usr/bin');"
        cmd = cmd + "os.system('gvim .toto.txt')";
        IPython.notebook.kernel.execute(cmd);
        return false;
    }}
);

// The 'u' shortcut.
IPython.keyboard_manager.command_shortcuts.add_shortcut('u', {
    handler : function (event) {
        function handle_output(msg) {
            var ret = msg.content.text;
            IPython.notebook.get_selected_cell().set_text(ret);
        }
        var callback = {'output': handle_output};
        var cmd = "f = open('.toto.txt', 'r');print(f.read())";
        IPython.notebook.kernel.execute(cmd, {iopub: callback},
                                        {silent: false});
        return false;
    }}
);

// vim: set filetype=javascript tw=75 sw=4:



The following cell brings in various python libraries.

 - `IPython.display` allows the viewing of an SVG image in the output of a
   code cell.

 - `ipywidgets` allows embedding a widget in the output of a code cell.

 - `bqplot` allows plotting with Bloomberg's high-performance, interacting
   plotting library.

 - `numpy` allows easy operations on arrays of data to be plotted.

 - `svgwrite` allows drawing an SVG image via Python code.

<!--- vim: set filetype=markdown tw=75: -->



In [ ]:
from IPython         import display
from IPython.display import SVG
from ipywidgets      import *
from bqplot          import *

import numpy         as np
import svgwrite      as sw

# vim: set tw=75:



## Interactive Drawing

An SVG drawing has an odd coordinate system.
 - The origin is at the *upper*-left corner of the drawing.
 - Each pair of coordinates is in the order (horizontal, vertical)
 - The more positive vertical coordinate is farther *down* the drawing.

The `svgwrite` module allows drawing from Python code.

The `drawing` function below takes a couple of parameters and draws a
simple object on the basis of them.

<!--- vim: set filetype=markdown tw=75: -->



In [ ]:
def drawing(angle, color):
    """
    Use the svgwrite module to draw a radial line of a certain color.

    Parameters:
    angle -- Angle (degrees) of radial line.
    color -- String for color of line.
    """

    sz = 200 # Size (pixels) of drawing.
    dr = sw.Drawing("drawing.svg", (sz, sz))
    r  = 0.5*sz            # Length of radial line.
    cx = 0.5*sz            # X-coordinate of center.
    cy = 0.5*sz            # Y-coordinate of center.
    a  = angle*np.pi/180.0 # Angle (radians).
    px = cx + r*np.cos(a)
    py = cy + r*np.sin(a)

    dr.add(dr.line((cx,cy), (px,py), stroke=color, stroke_width=3))

    # Return SVG object.
    return SVG(dr.tostring())



Then we call the `interact()` function defined by ipywidgets.

 - It produces a widget for each parameter taken by `drawing()`.

 - The name of each parameter, as it appears in the definition of
   `drawing()`, is initialized by a tuple that initializes the widget.

<!--- vim: set filetype=markdown tw=75: -->



In [ ]:
interact(drawing,
         angle=(0.0, 360.0, 0.1),
         color=['red', 'green', 'blue', 'cyan', 'magenta', 'yellow'])

# vim: set filetype=python tw=75:



## Interactive Plot

bqplot comes with a default toolbar that allows translating and scaling the
display of a plot.

The plot is efficiently updated whenever any of the remaining parameters is
modified by the way of a slider below the plot.

We begin by using `np.linspace()` to generate a range for values of the
horizontal coordinate.

Then we apply a transformation to make the vertical coordinates of the
curve to plot.

<!--- vim: set filetype=markdown tw=75: -->



In [ ]:

x = np.linspace(0, 1, 250)
y = x*x  # Initialize to simple parabola.

# vim: set filetype=python tw=75:



Next, we use bqplot to generate a graph.

In [ ]:
sc_x = LinearScale()
sc_y = LinearScale()

x_ax  = Axis(label='x', scale=sc_x)

y_ax = Axis(label='y',
            orientation='vertical',
            label_color='blue',
            color='#4040C0',
            scale=sc_y)

line = Lines(x=x, y=[y],
             scales={'x': sc_x, 'y': sc_y},
             colors=['#0000FF'])

fig = Figure(axes=[x_ax, y_ax], marks=[line])
fig.layout.width  = '600px'
fig.layout.height = '300px'
tb  = Toolbar(figure=fig)

VBox([fig, tb])



Next, we again use bqplot's `interact()` function to construct the
relevant sliders interactively to modify the graph.

In [ ]:
def update(a, b, c):
    line.y = a + b*x + c*x*x
    return None

interact(update,
         a=(-1.0, 1.0, 0.01),
         b=(-1.0, 1.0, 0.01),
         c=(-1.0, 1.0, 0.01))

